In [61]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [62]:
tracker=Tracker()
count=0

In [63]:
import cv2

def reduce_frame(input_video, output_video, keep_every_n_frame):
    cap = cv2.VideoCapture(input_video)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Déterminez le nombre de frames à conserver
    num_frames_to_keep = total_frames // keep_every_n_frame

    # Définir le codec et le writer pour la nouvelle vidéo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Écriture de la frame toutes les 'keep_every_n_frame' frames
        if count % keep_every_n_frame == 0:
            out.write(frame)

        count += 1
        if count >= num_frames_to_keep * keep_every_n_frame:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Utilisation de la fonction
input_video = 'cars.mp4'
output_video = 'cars_mod.mp4'
keep_every_n_frame = 2  # Vous pouvez ajuster cela en fonction de vos besoins

reduce_frame(input_video, output_video, keep_every_n_frame)


In [64]:
cap=cv2.VideoCapture('cars_mod.mp4')

down={}
counter_down=set()
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1

    results=model.predict(frame)

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    #print(px)
    
    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])


    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw ceter points of bounding box
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        # cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



        y = 308
        offset = 7
    
        ''' condition for red line '''
        if y < (cy + offset) and y > (cy - offset):
          ''' this if condition is putting the id and the circle on the object when the center of the object touched the red line.'''
          
          down[id]=cy   #cy is current position. saving the ids of the cars which are touching the red line first. 
          #This will tell us the travelling direction of the car.
          if id in down:         
           cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
           cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
           counter_down.add(id) 

    # # line
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    
    # print(down)
    #cv2.line(frame,(282,408),(1004,408),red_color,2)  #  starting cordinates and end of line cordinates
    #cv2.putText(frame,('red line'),(280,408),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA) 
    cv2.line(frame,(0,320),(1004,320),red_color,2)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(0,320),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA) 


    downwards = (len(counter_down))
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1, cv2.LINE_AA) 
    


    # cv2.line(frame,(282,308),(1004,308),red_color,3)  #  starting cordinates and end of line cordinates
    # cv2.putText(frame,('red line'),(280,308),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    
    
    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

0: 384x640 (no detections), 684.5ms
Speed: 2.7ms preprocess, 684.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 672.9ms
Speed: 3.0ms preprocess, 672.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 690.2ms
Speed: 3.0ms preprocess, 690.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 662.3ms
Speed: 2.0ms preprocess, 662.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 541.1ms
Speed: 2.0ms preprocess, 541.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 628.6ms
Speed: 3.1ms preprocess, 628.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 600.9ms
Speed: 3.2ms preprocess, 600.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 (no detections), 543.1ms
Speed: 2.0ms preprocess, 5